# Opdracht 2

1. Let's get the CSV-file in jupyter
    1. Let's clean the data
        1. Correct the datatypes
        2. Work on the duplicates
        3. Work on outliers
        4. Work on missing values
            1. Does all the data make sense (verifying data with statistics)
    2. Doing analysis

# 1. Lets get the first CSV-file in jupyter

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
#import ipynb.fs.defs.functions2 as enzo

# Remove restrictions on amount of rows and columns that can be displayed in pandas dataframes.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data = pd.read_csv("data/US_Accidents_June20.csv")

## A. Let's clean the data

## a. Correct the datatypes

In [2]:
# Store datetime columns in datetime format
data['parsed_starttime'] = pd.to_datetime(data['Start_Time'], format='%Y-%m-%d %H:%M:%S')
data['parsed_endtime'] = pd.to_datetime(data['End_Time'], format='%Y-%m-%d %H:%M:%S')
data['distance_km'] = data['Distance(mi)'] * 1.60934

#State (part II)
data['State'] = data['State'].astype('category') #transform the column State into the datatype category

#Zipcode
data['Zipcode'] = data['Zipcode'].astype('category') #transform the column zipcode into the datatype category

#Country
data.pop('Country') #we can exclude this variable as it all contains US as the whole dataset contains info from the US

#Turning_Loop
data.pop('Turning_Loop') #we can exclude this variable as it all contains False

#Timezone
data['Timezone'] = data['Timezone'].astype('category') #transform the column timezone into the datatype category

#Airport_Code
data['Airport_Code'] = data['Airport_Code'].astype('category') #transform the column Airport_code into the datatype category

#Weather_Timestamp
data['Weather_Timestamp'] = pd.to_datetime(data['Weather_Timestamp'], format='%Y-%m-%d %H:%M:%S')

#Temperature 
data['Temperature(C)'] = ((data['Temperature(F)'] - 32) * (5/9)) #this is the calculation that transforms the fahrenheit to celsius.

#Wind_Chill(F) 
data['Wind_Chill(C)'] = ((data['Wind_Chill(F)'] - 32) * (5/9)) #this is the calculation that transforms the fahrenheit to celsius.

#Pressure(in) - 
data['Pressure(in)'] = data['Pressure(in)'].astype('float') #lets first convert the pressure to float in order to do calculations
data['Pressure(hPa)'] = (data['Pressure(in)'] * 33.86389) #this is the calculation that transforms it from inHg to hPa

#Visibility(mi)
data['Visibility(mi)'] = data['Visibility(mi)'].astype('float') #lets first convert the pressure to float in order to do calculations
data['Visibility(km)'] = (data['Visibility(mi)'] * 1609.344) #this is the calculation that transforms the mi to km.

#Wind_Direction
data['Wind_Direction'].fillna('nan', inplace=True) #transform the NaN to 'NaN'(string) as it is giving problems in further fixing Wind_Direction 
data['Wind_Direction'] = data['Wind_Direction'].str.lower() #in case of differnences lower/higher cases between categories, we can make every category description in lowercase letters
mapping = {'east':'e', 'north':'n', 'south':'s', 'west':'w', 'variable':'other', 'var':'other'} #let's recategorize the potential categories
data['Wind_Direction'] = data['Wind_Direction'].replace(mapping)  #this allows us to replace the transformations stored in the variable mapping
data['Wind_Direction'] = data['Wind_Direction'].replace('nan', np.NaN, regex=True) #lets transform 'NaN' (string) back to NaN and give the outcome 'var' also the NaN-value

#Wind_Speed(mph) 
data['Wind_Speed(kph)'] = (data['Wind_Speed(mph)'] * 1.60934) #this does to trick in transforming the data from mph to kph

#Precipitation(in) 
data['Precipitation(cm)'] = (data['Precipitation(in)'] * 2.54) #this does the trick from inches to cm

#Weather_Condition 
data['Weather_Condition'] = data['Weather_Condition'].astype('category') #transform the column Airport_code into the datatype category

#lets loose the variables that we transformed, which are Temperature(F), Wind_Chill(F), Pressure(in), Visibility(mi), Wind_Speed(mph), Precipitation(in)
data = data.drop(['Temperature(F)', 'Wind_Chill(F)','Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)'], axis = 1)

## b. Work on the duplicates

In [3]:
# Remove duplicate rows
data.drop_duplicates(inplace = True)
#data

## c. Work on outliers

In [ ]:
#Build boxplot - temperature 
sns.boxplot(x=data['Temperature(C)'])  #highest recorded temperature in US is 56.7 celsius, the lowest recorderd temperature −56.7. Those are then the limits which will be maintained
data.loc[data['Temperature(C)'] > 56.7] = np.nan #replace values where it is higher than 56.7 with NAN's 
data.loc[data['Temperature(C)'] < -56.7] = np.nan #replace values where it is lower than -56.7 with NAN's 

#Build boxplot - Pressure(hPa) (Part I) - in hold
sns.boxplot(x=data['Pressure(hPa)']) #it seems we have one observation around 2000 and 0 which seems very high and low compared ot the rest of the values so let's chech them out first - air pressure can be affected by altitude, temperature and humidity

#Build boxplot - Wind_Speed(kph)
sns.boxplot(x=data['Wind_Speed(kph)'])
data.loc[data['Wind_Speed(kph)'] > 372] = np.nan #highest windspeed ever recorded in USA is 371.76 kph so everything above is, is not possible

#Build boxplot - Precipitation(cm)
sns.boxplot(x=data['Precipitation(cm)']) 
data.loc[data['Wind_Speed(kph)'] > 50] = np.nan #we don't find the values above 50 cm feasable as the highest amount of daily precipitation is for tennesse 21.082 cm (https://www.afsrepair.com/resources/rainiest-cities-towns-in-tennessee/) and for california is that 0.3302 cm in long beach which is around 15 km from the site (https://www.dailybreeze.com/2021/10/26/new-rainfall-records-set-at-lax-and-long-beach-airport/)

## d. Work on missing values

In [ ]:
#lets check the relative share of NA's within each column
((data.isnull() | data.isna()).sum() * 100 / data.index.size).round(2)

### i. Does all the data make sense (verifying data with statistics)

# B. Doing analysis

In [4]:
data.to_csv("parsed_data.csv", index = False)

## A. Exploratory Data Analysis

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5, 10)

In [ ]:
# Scatterplot -temperature + visibility grouped by severity 
sns.relplot(
    data=data,
    x="Temperature(C)", y="Visibility(km)",
    hue="Severity", style= "Severity"
)
# When visibility is low, around 0C and -25C are clusters of very severe accidents

In [ ]:
# hist -distance groupded by severity
sns.histplot(
    data=data,
    x="Distance(mi)",
    hue="Severity", multiple='stack'
)

In [ ]:
# show the number of observations in each severe levels while grouped by Stop.
sns.catplot(x="Severity", hue="Stop", kind="count",
            data=data)

In [ ]:
# show the number of observations in each severe levels while grouped by Traffic_Calming.
sns.catplot(x="Severity", hue="Traffic_Calming", kind="count",
            data=data)

In [ ]:
# show the number of observations in each severe levels while grouped by Traffic_Signal.
sns.catplot(x="Severity", hue="Traffic_Signal", kind="count",
            data=data)

In [ ]:
# show the number of observations in each severe levels while grouped by Bump.
sns.catplot(x="Severity", hue="Bump", kind="count",
            data=data)

In [ ]:
# show the number of observations in each severe levels while grouped by Junction.
sns.catplot(x="Severity", hue="Junction", kind="count",
            data=data)

In [ ]:
# show the number of observations in each severe levels while grouped by Give_Way.
sns.catplot(x="Severity", hue="Give_Way", kind="count",
            data=data)